In [3]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np

In [4]:
#Importamos los sets de las películas
amazon_titles = pd.read_csv('C:/Users/larra/OneDrive/Documents/HENRY/PI01/MLOpsReviews/amazon_prime_titles.csv')
disney_titles = pd.read_csv('C:/Users/larra/OneDrive/Documents/HENRY/PI01/MLOpsReviews/disney_plus_titles.csv')
hulu_titles = pd.read_csv('C:/Users/larra/OneDrive/Documents/HENRY/PI01/MLOpsReviews/hulu_titles.csv')
netflix_titles = pd.read_csv('C:/Users/larra/OneDrive/Documents/HENRY/PI01/MLOpsReviews/netflix_titles.csv')

In [5]:
#Hacemos una primera inspección
amazon_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [6]:
amazon_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

In [7]:
'''Vemos que no hay no nulos pero tambien debemos asegurarnos que los Id sean unicos.
Definimos funcion para controlar que no haya repetidos en los campos que interesen.'''
def controlRepetidos (dataframe,campo):
    contador = 0
    for i in dataframe[campo].value_counts():
        if i != 1:
            contador += 1
    print('Se repiten '+str(contador)+' valores mas de una vez')

In [8]:
# Vamos a ver si se repiten los valores bajo showId.
controlRepetidos(amazon_titles,'show_id')

Se repiten 0 valores mas de una vez


In [9]:
#Definimos la funcion para agregar la letra a la columna showId
def agregarLetraId (dataframe,letra):
    for i in range(dataframe.shape[0]):
        dataframe['show_id'][i]=letra + dataframe['show_id'][i]
    return dataframe

In [10]:
#Agregamos la letra en el dataset de amazon.
agregarLetraId(amazon_titles,'a')
amazon_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\1232259299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['show_id'][i]=letra + dataframe['show_id'][i]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,as2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,as3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,as4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,as5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

In [11]:
#Vemos que tipo de valores hay en la columna rating
amazon_titles['rating'].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

In [12]:
'''Hay muchas formas de llamar al "general for all audiences" (all,all_ages,Tv-pg,g).
Esto debiera unificarse pero nos pidieron que solamente completaramos los faltantes con "G".'''
amazon_titles['rating'].fillna('G',inplace=True)
amazon_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,as2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,as3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,G,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,as4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,G,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,as5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,G,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


De haber fechas, deberán tener el formato AAAA-mm-dd

In [13]:
#Vemos que valores de fecha encontramos bajo esta columna
amazon_titles['date_added'].unique()

array(['March 30, 2021', 'April 1, 2021', 'April 4, 2021',
       'April 10, 2021', 'April 17, 2021', 'April 24, 2021',
       'May 2, 2021', 'June 3, 2021', nan, 'April 13, 2021',
       'April 20, 2021', 'April 26, 2021', 'April 27, 2021',
       'April 30, 2021', 'May 1, 2021', 'May 9, 2021', 'May 11, 2021',
       'May 12, 2021', 'May 13, 2021', 'May 18, 2021', 'May 19, 2021',
       'May 22, 2021', 'May 24, 2021', 'May 29, 2021', 'June 4, 2021',
       'June 11, 2021', 'June 12, 2021', 'June 15, 2021', 'June 18, 2021',
       'June 20, 2021', 'June 21, 2021', 'June 22, 2021', 'June 23, 2021',
       'June 24, 2021', 'June 25, 2021', 'June 26, 2021', 'June 29, 2021',
       'July 1, 2021', 'July 3, 2021', 'July 8, 2021', 'July 12, 2021',
       'July 16, 2021', 'July 17, 2021', 'July 20, 2021', 'July 23, 2021',
       'July 24, 2021', 'July 26, 2021', 'July 27, 2021',
       'August 1, 2021', 'August 4, 2021', 'August 6, 2021',
       'August 14, 2021', 'August 15, 2021', 'August 1

In [14]:
amazon_titles['date_added'].unique().size

85

In [15]:
#Son 85 valores. 84 fechas distintas y el valor nulo repetido mas de 9.000 veces. Le asignamos dtype = datetime.
amazon_titles = amazon_titles.astype({'date_added':'datetime64[D]'})
amazon_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,G,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,as2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,as3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,G,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,as4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2021-03-30,2014,G,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,as5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,2021-03-30,1989,G,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [16]:
#Vemos que se haya cambiado el tipo de dato
amazon_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       9668 non-null   object        
 1   type          9668 non-null   object        
 2   title         9668 non-null   object        
 3   director      7586 non-null   object        
 4   cast          8435 non-null   object        
 5   country       672 non-null    object        
 6   date_added    155 non-null    datetime64[ns]
 7   release_year  9668 non-null   int64         
 8   rating        9668 non-null   object        
 9   duration      9668 non-null   object        
 10  listed_in     9668 non-null   object        
 11  description   9668 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 906.5+ KB


In [17]:
#Chequeamos los valores unicos y vemos que se encuentre el valor nulo entre ellos.
amazon_titles['date_added'].unique()

array(['2021-03-30T00:00:00.000000000', '2021-04-01T00:00:00.000000000',
       '2021-04-04T00:00:00.000000000', '2021-04-10T00:00:00.000000000',
       '2021-04-17T00:00:00.000000000', '2021-04-24T00:00:00.000000000',
       '2021-05-02T00:00:00.000000000', '2021-06-03T00:00:00.000000000',
                                 'NaT', '2021-04-13T00:00:00.000000000',
       '2021-04-20T00:00:00.000000000', '2021-04-26T00:00:00.000000000',
       '2021-04-27T00:00:00.000000000', '2021-04-30T00:00:00.000000000',
       '2021-05-01T00:00:00.000000000', '2021-05-09T00:00:00.000000000',
       '2021-05-11T00:00:00.000000000', '2021-05-12T00:00:00.000000000',
       '2021-05-13T00:00:00.000000000', '2021-05-18T00:00:00.000000000',
       '2021-05-19T00:00:00.000000000', '2021-05-22T00:00:00.000000000',
       '2021-05-24T00:00:00.000000000', '2021-05-29T00:00:00.000000000',
       '2021-06-04T00:00:00.000000000', '2021-06-11T00:00:00.000000000',
       '2021-06-12T00:00:00.000000000', '2021-06-15

In [18]:
#Chequeamos cuantos valores unicos hay y vemos que coincida con los 85 que habia.
amazon_titles['date_added'].unique().size

85

Los campos de texto deberán estar en minúsculas, sin excepciones

In [19]:
#Definimos la funcion para colocar los campos de texto en minusculas.
def textoEnMinusculas(dataframe):
    columns = ['type','title','director','cast','country','rating','duration','listed_in','description']
    for col in columns:
        for i in range(dataframe.shape[0]):
            try:
                dataframe[col][i] = dataframe[col][i].casefold()
            except:
                pass
    return dataframe

In [20]:
#Modificamos las columnas de texto a minusculas.
textoEnMinusculas(amazon_titles)
amazon_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\3644383547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col][i] = dataframe[col][i].casefold()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,"comedy, drama",a small fishing village must procure a local d...
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,"drama, international",a metro family decides to fight a cyber crimin...
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,"action, drama, suspense",after a man discovers his wife is cheating on ...
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,documentary,"pink breaks the mold once again, bringing her ..."
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,"drama, fantasy",teenage matt banting wants to work with a famo...


El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [21]:
#Definimos la funcion para separar la duracion en su numero y la medida (minutos o seasons)
def divisionDuration (dataframe):
    cantidades = []
    medidas = []
    for i in range(dataframe.shape[0]):
        try:
            cant, medida = dataframe['duration'][i].split()
            cantidades.append(int(cant))
            medidas.append(medida)
        except:
            cantidades.append(np.NaN)
            medidas.append(np.NaN)
    try:
        dataframe.insert(10,column='duration.int',value=cantidades)
        dataframe.insert(11,column='duration.type',value=medidas)
        return dataframe
    except:
        return 'Ya existen columnas con esos nombres'

In [22]:
#Dividimos el campo duration del df de amazon
divisionDuration(amazon_titles)
amazon_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration.int,duration.type,listed_in,description
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113 min,113,min,"comedy, drama",a small fishing village must procure a local d...
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110 min,110,min,"drama, international",a metro family decides to fight a cyber crimin...
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74 min,74,min,"action, drama, suspense",after a man discovers his wife is cheating on ...
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69 min,69,min,documentary,"pink breaks the mold once again, bringing her ..."
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45 min,45,min,"drama, fantasy",teenage matt banting wants to work with a famo...


In [23]:
#Quitamos la columna duration.
amazon_titles.drop(columns='duration',inplace=True)

In [24]:
#Observamos los valores que quedan en duration.type
amazon_titles['duration.type'].unique()

array(['min', 'season', 'seasons'], dtype=object)

In [25]:
#Unificamos Seasons y Season.
amazon_titles['duration.type'].replace(to_replace='season',value='seasons',inplace=True)
amazon_titles['duration.type'].unique()

array(['min', 'seasons'], dtype=object)

Ahora debemos hacer lo mismo pero con los dataframes de las otros servicios de streaming

In [26]:
disney_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [27]:
disney_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [28]:
#Vemos que no hay nulos bajo show_id. Controlamos que no se repitan.
controlRepetidos(disney_titles,'show_id')

Se repiten 0 valores mas de una vez


In [29]:
#Agregamos la letra en show_id.
agregarLetraId(disney_titles,'d')
disney_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\1232259299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['show_id'][i]=letra + dataframe['show_id'][i]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ds1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,ds2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,ds3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,ds4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,ds5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [30]:
#Vemos que tipo de valores hay en la columna rating
disney_titles['rating'].unique()

array(['TV-G', 'PG', 'TV-PG', nan, 'PG-13', 'TV-14', 'G', 'TV-Y7', 'TV-Y',
       'TV-Y7-FV'], dtype=object)

In [31]:
#Nuevamente. Debiera normalizarse y elegir uno solo. De momento solo se llena con G.
disney_titles['rating'].fillna('G',inplace=True)
disney_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ds1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,ds2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,ds3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,ds4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,ds5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,G,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [32]:
#Vemos que se encuentra bajo la columna date_added.
disney_titles['date_added'].unique()

array(['November 26, 2021', 'November 25, 2021', 'November 24, 2021',
       'November 19, 2021', 'November 17, 2021', 'November 12, 2021',
       'November 5, 2021', 'November 3, 2021', 'October 29, 2021',
       'October 22, 2021', 'October 20, 2021', 'October 15, 2021',
       'October 13, 2021', 'October 8, 2021', 'October 6, 2021',
       'October 1, 2021', 'September 29, 2021', 'September 24, 2021',
       'September 22, 2021', 'September 17, 2021', 'September 15, 2021',
       'September 10, 2021', 'September 8, 2021', 'September 3, 2021',
       'September 1, 2021', 'August 27, 2021', 'August 25, 2021',
       'August 20, 2021', 'August 18, 2021', 'August 13, 2021',
       'August 11, 2021', 'August 6, 2021', 'August 4, 2021',
       'July 30, 2021', 'July 28, 2021', 'July 23, 2021', 'July 21, 2021',
       'July 16, 2021', 'July 14, 2021', 'July 9, 2021', 'July 7, 2021',
       'July 2, 2021', 'June 25, 2021', 'June 18, 2021', 'June 11, 2021',
       'June 9, 2021', 'June 4, 2

In [33]:
#Vemos cuantas son.
disney_titles['date_added'].unique().size

168

In [34]:
#Modificamos el formato de las fechas.
disney_titles = disney_titles.astype({'date_added':'datetime64[D]'})
disney_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ds1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,2021-11-26,2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,ds2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,2021-11-26,1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,ds3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,2021-11-26,2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,ds4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,2021-11-26,2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,ds5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,2021-11-25,2021,G,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [35]:
#Vemos que se haya modificado.
disney_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       1450 non-null   object        
 1   type          1450 non-null   object        
 2   title         1450 non-null   object        
 3   director      977 non-null    object        
 4   cast          1260 non-null   object        
 5   country       1231 non-null   object        
 6   date_added    1447 non-null   datetime64[ns]
 7   release_year  1450 non-null   int64         
 8   rating        1450 non-null   object        
 9   duration      1450 non-null   object        
 10  listed_in     1450 non-null   object        
 11  description   1450 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 136.1+ KB


In [36]:
#Corroboramos que coincida.
print(disney_titles['date_added'].unique().size)
print(disney_titles['date_added'].unique())

168
['2021-11-26T00:00:00.000000000' '2021-11-25T00:00:00.000000000'
 '2021-11-24T00:00:00.000000000' '2021-11-19T00:00:00.000000000'
 '2021-11-17T00:00:00.000000000' '2021-11-12T00:00:00.000000000'
 '2021-11-05T00:00:00.000000000' '2021-11-03T00:00:00.000000000'
 '2021-10-29T00:00:00.000000000' '2021-10-22T00:00:00.000000000'
 '2021-10-20T00:00:00.000000000' '2021-10-15T00:00:00.000000000'
 '2021-10-13T00:00:00.000000000' '2021-10-08T00:00:00.000000000'
 '2021-10-06T00:00:00.000000000' '2021-10-01T00:00:00.000000000'
 '2021-09-29T00:00:00.000000000' '2021-09-24T00:00:00.000000000'
 '2021-09-22T00:00:00.000000000' '2021-09-17T00:00:00.000000000'
 '2021-09-15T00:00:00.000000000' '2021-09-10T00:00:00.000000000'
 '2021-09-08T00:00:00.000000000' '2021-09-03T00:00:00.000000000'
 '2021-09-01T00:00:00.000000000' '2021-08-27T00:00:00.000000000'
 '2021-08-25T00:00:00.000000000' '2021-08-20T00:00:00.000000000'
 '2021-08-18T00:00:00.000000000' '2021-08-13T00:00:00.000000000'
 '2021-08-11T00:00:00

In [37]:
#Modificamos las columnas de texto a minusculas.
textoEnMinusculas(disney_titles)
disney_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\3644383547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col][i] = dataframe[col][i].casefold()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ds1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",NaN,2021-11-26,2016,tv-g,23 min,"animation, family",join mickey and the gang as they duck the halls!
1,ds2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",NaN,2021-11-26,1988,pg,91 min,comedy,santa claus passes his magic bag to a new st. ...
2,ds3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,"animation, comedy, family",sid the sloth is on santa's naughty list.
3,ds4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",NaN,2021-11-26,2021,tv-pg,41 min,musical,"this is real life, not just fantasy!"
4,ds5,tv show,the beatles: get back,NaN,"john lennon, paul mccartney, george harrison, ...",NaN,2021-11-25,2021,g,1 season,"docuseries, historical, music",a three-part documentary from peter jackson ca...


In [38]:
#Vemos que hay debajo del campo duration
disney_titles['duration'].unique()

array(['23 min', '91 min', '41 min', '1 season', '94 min', '2 seasons',
       '45 min', '59 min', '4 min', '10 seasons', '2 min', '1 min',
       '8 min', '3 seasons', '110 min', '11 min', '7 min', '95 min',
       '129 min', '14 min', '5 min', '134 min', '9 min', '30 min',
       '6 min', '22 min', '3 min', '87 min', '90 min', '93 min',
       '101 min', '132 min', '44 min', '131 min', '104 min', '5 seasons',
       '52 min', '47 min', '53 min', '85 min', '92 min', '135 min',
       '4 seasons', '105 min', '99 min', '32 seasons', '88 min',
       '127 min', '106 min', '116 min', '42 min', '16 seasons', '140 min',
       '19 seasons', '66 min', '115 min', '137 min', '9 seasons',
       '126 min', '82 min', '125 min', '54 min', '7 seasons', '98 min',
       '102 min', '80 min', '170 min', '97 min', '112 min', '21 min',
       '20 min', '15 min', '19 min', '79 min', '6 seasons', '89 min',
       '18 min', '81 min', '148 min', '121 min', '96 min', '109 min',
       '107 min', '86 min', '

In [39]:
#No hay nulos y los valores estan todos normales. Hacemos la division.
divisionDuration(disney_titles)
disney_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration.int,duration.type,listed_in,description
0,ds1,movie,duck the halls: a mickey mouse christmas special,"alonso ramirez ramos, dave wasson","chris diamantopoulos, tony anselmo, tress macn...",NaN,2021-11-26,2016,tv-g,23 min,23,min,"animation, family",join mickey and the gang as they duck the halls!
1,ds2,movie,ernest saves christmas,john cherry,"jim varney, noelle parker, douglas seale",NaN,2021-11-26,1988,pg,91 min,91,min,comedy,santa claus passes his magic bag to a new st. ...
2,ds3,movie,ice age: a mammoth christmas,karen disher,"raymond albert romano, john leguizamo, denis l...",united states,2021-11-26,2011,tv-g,23 min,23,min,"animation, comedy, family",sid the sloth is on santa's naughty list.
3,ds4,movie,the queen family singalong,hamish hamilton,"darren criss, adam lambert, derek hough, alexa...",NaN,2021-11-26,2021,tv-pg,41 min,41,min,musical,"this is real life, not just fantasy!"
4,ds5,tv show,the beatles: get back,NaN,"john lennon, paul mccartney, george harrison, ...",NaN,2021-11-25,2021,g,1 season,1,season,"docuseries, historical, music",a three-part documentary from peter jackson ca...


In [40]:
#Quitamos la columna duration
disney_titles.drop(columns='duration',inplace=True)

In [41]:
#Observamos los valores que quedan en duration.type
disney_titles['duration.type'].unique()

array(['min', 'season', 'seasons'], dtype=object)

In [42]:
#Unificamos Seasons y Season.
disney_titles['duration.type'].replace(to_replace='season',value='seasons',inplace=True)
disney_titles['duration.type'].unique()

array(['min', 'seasons'], dtype=object)

In [43]:
#Ahora lo hacemos con hulu.
hulu_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [44]:
hulu_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [45]:
#Vemos que no hay nulos bajo show_id. Controlamos que no se repitan.
controlRepetidos(hulu_titles,'show_id')

Se repiten 0 valores mas de una vez


In [46]:
#Agregamos la letra en show_id.
agregarLetraId(hulu_titles,'h')
hulu_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\1232259299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['show_id'][i]=letra + dataframe['show_id'][i]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,hs1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,hs2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,hs3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,hs4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,hs5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [47]:
#Vemos que tipo de valores hay en la columna rating
hulu_titles['rating'].unique()

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', '2 Seasons', 'TV-Y', '93 min', '4 Seasons', 'TV-Y7',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 Seasons', '121 min', '88 min', '101 min', '1 Season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '84 min', '103 min', '105 min', '119 min', '114 min', '82 min',
       '90 min', '130 min', '110 min', '80 min', '6 Seasons', '97 min',
       '111 min', '81 min', '49 min', '45 min', '41 min', '73 min',
       '40 min', '36 min', '39 min', '34 min', '47 min', '65 min',
       '37 min', '78 min', '102 min', '129 min', '115 min', '112 min',
       'NR', '61 min', '106 min', '76 min', '77 min', '79 min', '157 min',
       '28 min', '64 min', '7 min', '5 min', '6 min', '127 min',
       '142 min', '108 min', '57 min', '118 min', '116 min', '12 Seasons',
       '71 

In [48]:
#En este caso se ve que muchos valores de duration pasó a estar debajo de rating. Vamos a revisar uno al azar.
hulu_titles[hulu_titles['rating']=='2 Seasons']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
50,hs51,Movie,Yashahime: Princess Half-Demon,NaN,NaN,Japan,"October 9, 2021",2020,2 Seasons,NaN,Anime,Long-lost twins Towa and Setsuna reunite after...
471,hs472,Movie,This Way Up,NaN,NaN,United Kingdom,"July 9, 2021",2019,2 Seasons,NaN,"Comedy, Drama, International","This Way Up is a comedy drama about moving on,..."
801,hs802,Movie,Everything's Gonna Be Okay,NaN,NaN,United States,"April 9, 2021",2020,2 Seasons,NaN,"Comedy, Drama",Australian twenty-something Nicholas is left t...
847,hs848,Movie,Juda,NaN,NaN,Israel,"March 30, 2021",2017,2 Seasons,NaN,"Action, Adventure, Crime",Juda is a low-life gambler hustling a living i...
1135,hs1136,Movie,Brassic,NaN,NaN,United Kingdom,"December 4, 2020",2019,2 Seasons,NaN,Comedy,"Vinnie, a Lancashire lad with bipolar disorder..."
1464,hs1465,Movie,El embarcadero,NaN,NaN,Spain,"August 1, 2020",2019,2 Seasons,NaN,"Drama, Horror, Mystery",High-profile architect Alexandra faces her wor...
1587,hs1588,Movie,Your Home Made Perfect,NaN,NaN,United Kingdom,"June 15, 2020",2019,2 Seasons,NaN,Reality,Never before have plans come to life like this...
2270,hs2271,Movie,False Flag,NaN,NaN,Israel,"February 14, 2019",2015,2 Seasons,NaN,"Action, Adventure, Drama",Five Israeli citizens find themselves plunged ...
2908,hs2909,Movie,The Wrong Mans,NaN,NaN,United Kingdom,"December 24, 2014",2013,2 Seasons,NaN,"Action, Adventure, Comedy",Lowly office workers Sam (Matthew Baynton) and...


In [49]:
#Efectivamente ese parece ser el caso.Definimos funcion para hallar elementos de duracion dentro del campo rating y examinarlos.
def duracionEnRating (dataframe,cond):
    auxiliar = []
    for i in range(dataframe['rating'].unique().size):
        if type(dataframe['rating'].unique()[i])==str and dataframe['rating'].unique()[i].find(cond)!=-1:
            auxiliar.append(dataframe['rating'].unique()[i])
    return dataframe[dataframe['rating'].isin(auxiliar)]

In [50]:
#Examinamos todas las que tienen min.
duracionEnRating(hulu_titles,'min')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
108,hs109,Movie,Maze,NaN,NaN,"Ireland, United Kingdom, Germany, Sweden","October 1, 2021",2017,93 min,NaN,"Action, Adventure, Crime",Based on the incredible true story of the 1983...
397,hs398,Movie,Gunfight at the O.K. Corral,NaN,NaN,United States,"August 1, 2021",1957,136 min,NaN,"Classics, Documentaries, Drama",Dramatization of the legendary battle between ...
495,hs496,Movie,78/52: Hitchcock's Shower Scene,NaN,NaN,United States,"July 1, 2021",2017,91 min,NaN,"Documentaries, Thriller",Just when you thought it was safe to go back i...
510,hs511,Movie,Citizen Jane: Battle for the City,NaN,NaN,United States,"July 1, 2021",2017,93 min,NaN,Documentaries,Chronicle of activist Jane Jacobs' battle with...
513,hs514,Movie,Dealt,NaN,NaN,United States,"July 1, 2021",2017,85 min,NaN,Documentaries,Prepare to be inspired and amazed by Richard T...
...,...,...,...,...,...,...,...,...,...,...,...,...
2785,hs2786,Movie,The Beatles: Eight Days a Week - The Touring Y...,NaN,NaN,"United States, United Kingdom","September 17, 2016",2016,106 min,NaN,"Documentaries, Music",Academy Award-winner Ron Howard’s highly antic...
2879,hs2880,Movie,The Colony,NaN,NaN,"Canada, United States","June 12, 2015",2013,94 min,NaN,"Action, Adventure, Horror","Years ago, winter came and left. Survivors hav..."
2907,hs2908,Movie,Ron White: A Little Unprofessional,NaN,NaN,United States,"January 1, 2015",2012,80 min,NaN,"Comedy, Stand Up",Ron White delivers more than an hour of all ne...
2958,hs2959,Movie,(Sub) Ninja Scroll,NaN,NaN,Japan,"December 15, 2013",1993,93 min,NaN,"Action, Adult Animation, Adventure",When Jubei saves a young ninja woman from the ...


In [51]:
#Son 230 filas. Pero probamos viendo si todos los valores de duration son nulos.
duracionEnRating(hulu_titles,'min').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 230 entries, 108 to 2993
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       230 non-null    object 
 1   type          230 non-null    object 
 2   title         230 non-null    object 
 3   director      0 non-null      object 
 4   cast          0 non-null      float64
 5   country       228 non-null    object 
 6   date_added    230 non-null    object 
 7   release_year  230 non-null    int64  
 8   rating        230 non-null    object 
 9   duration      0 non-null      object 
 10  listed_in     230 non-null    object 
 11  description   230 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 23.4+ KB


In [52]:
#Efectivamente es así. Definimos la función para pasar estos valores debajo de duration y llenar con nulos los de rating.
def cambioDeColumnas (dataframe,campo_desde,campo_hacia,cond):
    for i in range(dataframe.shape[0]):
        if type(dataframe[campo_desde][i])==str and dataframe[campo_desde][i].find(cond)!=-1:
            aux = dataframe[campo_hacia][i]
            dataframe[campo_hacia][i] = dataframe[campo_desde][i]
            dataframe[campo_desde][i] = aux
    return dataframe

In [53]:
#Realizamos el cambio. Chequeamos con algunos Id que sabiamos que estan mal.
cambioDeColumnas(hulu_titles,'rating','duration','min')
lista_ids = ['hs109','hs496','hs2994']
hulu_titles[hulu_titles['show_id'].isin(lista_ids)]

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_hacia][i] = dataframe[campo_desde][i]
C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_desde][i] = aux


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
108,hs109,Movie,Maze,NaN,NaN,"Ireland, United Kingdom, Germany, Sweden","October 1, 2021",2017,NaN,93 min,"Action, Adventure, Crime",Based on the incredible true story of the 1983...
495,hs496,Movie,78/52: Hitchcock's Shower Scene,NaN,NaN,United States,"July 1, 2021",2017,NaN,91 min,"Documentaries, Thriller",Just when you thought it was safe to go back i...
2993,hs2994,Movie,Amy Winehouse: A Final Goodbye,NaN,NaN,United Kingdom,"December 13, 2012",2011,NaN,71 min,"Documentaries, Music",A Final Goodbye is an intimate look into the l...


In [54]:
#Examinamos todas las que tienen Season.
duracionEnRating(hulu_titles,'Season')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
50,hs51,Movie,Yashahime: Princess Half-Demon,NaN,NaN,Japan,"October 9, 2021",2020,2 Seasons,NaN,Anime,Long-lost twins Towa and Setsuna reunite after...
263,hs264,Movie,Trolls: TrollsTopia,NaN,NaN,United States,"September 2, 2021",2020,4 Seasons,NaN,"Family, Kids",Trolls: TrollsTopia! is the next chapter in th...
471,hs472,Movie,This Way Up,NaN,NaN,United Kingdom,"July 9, 2021",2019,2 Seasons,NaN,"Comedy, Drama, International","This Way Up is a comedy drama about moving on,..."
801,hs802,Movie,Everything's Gonna Be Okay,NaN,NaN,United States,"April 9, 2021",2020,2 Seasons,NaN,"Comedy, Drama",Australian twenty-something Nicholas is left t...
847,hs848,Movie,Juda,NaN,NaN,Israel,"March 30, 2021",2017,2 Seasons,NaN,"Action, Adventure, Crime",Juda is a low-life gambler hustling a living i...
...,...,...,...,...,...,...,...,...,...,...,...,...
2908,hs2909,Movie,The Wrong Mans,NaN,NaN,United Kingdom,"December 24, 2014",2013,2 Seasons,NaN,"Action, Adventure, Comedy",Lowly office workers Sam (Matthew Baynton) and...
2926,hs2927,Movie,The Hotwives of Orlando,NaN,NaN,United States,"July 15, 2014",2014,1 Season,NaN,"Comedy, Sitcom",The reality genre parody examines the glamorou...
2951,hs2952,Movie,All Saints (1998),NaN,NaN,Australia,"February 19, 2014",1998,12 Seasons,NaN,"Drama, International",All Saints is an Australian medical drama focu...
2955,hs2956,Movie,Getting On,NaN,NaN,United Kingdom,"January 7, 2014",2009,3 Seasons,NaN,"Comedy, International, Sitcom",Care for the elderly is the least glamorous ar...


In [55]:
#Son 62 mas. Chequeamos que todas las de duration sean nulas.
duracionEnRating(hulu_titles,'Season').info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 50 to 2959
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       62 non-null     object 
 1   type          62 non-null     object 
 2   title         62 non-null     object 
 3   director      0 non-null      object 
 4   cast          0 non-null      float64
 5   country       62 non-null     object 
 6   date_added    62 non-null     object 
 7   release_year  62 non-null     int64  
 8   rating        62 non-null     object 
 9   duration      0 non-null      object 
 10  listed_in     62 non-null     object 
 11  description   62 non-null     object 
dtypes: float64(1), int64(1), object(10)
memory usage: 6.3+ KB


In [56]:
#Tambien las modificamos.
cambioDeColumnas(hulu_titles,'rating','duration','Season')
lista_ids = ['hs472','hs2927','hs51']
hulu_titles[hulu_titles['show_id'].isin(lista_ids)]

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_hacia][i] = dataframe[campo_desde][i]
C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_desde][i] = aux


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
50,hs51,Movie,Yashahime: Princess Half-Demon,NaN,NaN,Japan,"October 9, 2021",2020,NaN,2 Seasons,Anime,Long-lost twins Towa and Setsuna reunite after...
471,hs472,Movie,This Way Up,NaN,NaN,United Kingdom,"July 9, 2021",2019,NaN,2 Seasons,"Comedy, Drama, International","This Way Up is a comedy drama about moving on,..."
2926,hs2927,Movie,The Hotwives of Orlando,NaN,NaN,United States,"July 15, 2014",2014,NaN,1 Season,"Comedy, Sitcom",The reality genre parody examines the glamorou...


In [57]:
#Chequeamos que se hayan hecho todos los cambios (comparamos con el info anterior)
hulu_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2261 non-null   object 
 9   duration      2886 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [58]:
#Vemos que tipo de valores hay ahora en la columna rating
hulu_titles['rating'].unique()

array(['TV-MA', nan, 'PG-13', 'R', 'TV-14', 'PG', 'TV-PG', 'NOT RATED',
       'G', 'TV-G', 'TV-Y', 'TV-Y7', 'NR'], dtype=object)

In [59]:
#Nuevamente. Debiera normalizarse y elegir uno solo. De momento solo se llena con G.
hulu_titles['rating'].fillna('G',inplace=True)
hulu_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,hs1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,hs2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,G,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,hs3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,hs4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,hs5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,G,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [60]:
#Vemos que se encuentra bajo la columna date_added.
hulu_titles['date_added'].unique()

array(['October 24, 2021', 'October 23, 2021', 'October 22, 2021', ...,
       'June 1, 2008', 'July 14, 2006', nan], dtype=object)

In [61]:
#Vemos cuantas son.
hulu_titles['date_added'].unique().size

1116

In [62]:
#Modificamos el formato de las fechas.
hulu_titles = hulu_titles.astype({'date_added':'datetime64[D]'})
hulu_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,hs1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,2021-10-24,2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,hs2,Movie,Silent Night,NaN,NaN,NaN,2021-10-23,2020,G,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,hs3,Movie,The Marksman,NaN,NaN,NaN,2021-10-23,2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,hs4,Movie,Gaia,NaN,NaN,NaN,2021-10-22,2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,hs5,Movie,Settlers,NaN,NaN,NaN,2021-10-22,2021,G,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [63]:
#Vemos que se haya modificado.
hulu_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       3073 non-null   object        
 1   type          3073 non-null   object        
 2   title         3073 non-null   object        
 3   director      3 non-null      object        
 4   cast          0 non-null      float64       
 5   country       1620 non-null   object        
 6   date_added    3045 non-null   datetime64[ns]
 7   release_year  3073 non-null   int64         
 8   rating        3073 non-null   object        
 9   duration      2886 non-null   object        
 10  listed_in     3073 non-null   object        
 11  description   3069 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 288.2+ KB


In [64]:
#Corroboramos que coincida.
print(hulu_titles['date_added'].unique().size)
print(hulu_titles['date_added'].unique())


1116
['2021-10-24T00:00:00.000000000' '2021-10-23T00:00:00.000000000'
 '2021-10-22T00:00:00.000000000' ... '2008-06-01T00:00:00.000000000'
 '2006-07-14T00:00:00.000000000'                           'NaT']


In [65]:
#Modificamos las columnas de texto a minusculas.
textoEnMinusculas(hulu_titles)
hulu_titles.head()


C:\Users\larra\AppData\Local\Temp\ipykernel_5372\3644383547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col][i] = dataframe[col][i].casefold()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,hs1,movie,ricky velez: here's everything,NaN,NaN,NaN,2021-10-24,2021,tv-ma,NaN,"comedy, stand up",​comedian ricky velez bares it all with his ho...
1,hs2,movie,silent night,NaN,NaN,NaN,2021-10-23,2020,g,94 min,"crime, drama, thriller","mark, a low end south london hitman recently r..."
2,hs3,movie,the marksman,NaN,NaN,NaN,2021-10-23,2021,pg-13,108 min,"action, thriller",a hardened arizona rancher tries to protect an...
3,hs4,movie,gaia,NaN,NaN,NaN,2021-10-22,2021,r,97 min,horror,a forest ranger and two survivalists with a cu...
4,hs5,movie,settlers,NaN,NaN,NaN,2021-10-22,2021,g,104 min,"science fiction, thriller",mankind's earliest settlers on the martian fro...


In [66]:
#Vemos que hay debajo del campo duration
hulu_titles['duration'].unique()

array([nan, '94 min', '108 min', '97 min', '104 min', '1 season',
       '88 min', '3 seasons', '113 min', '6 seasons', '87 min',
       '7 seasons', '2 seasons', '4 seasons', '84 min', '95 min',
       '15 seasons', '101 min', '93 min', '105 min', '12 seasons',
       '8 seasons', '99 min', '112 min', '124 min', '23 seasons',
       '145 min', '125 min', '157 min', '41 min', '89 min', '5 seasons',
       '80 min', '90 min', '116 min', '102 min', '100 min', '110 min',
       '114 min', '115 min', '130 min', '133 min', '91 min', '121 min',
       '96 min', '107 min', '103 min', '109 min', '98 min', '119 min',
       '106 min', '111 min', '118 min', '117 min', '136 min', '181 min',
       '182 min', '128 min', '138 min', '126 min', '72 min', '16 seasons',
       '20 seasons', '9 seasons', '86 min', '75 min', '144 min', '26 min',
       '76 min', '129 min', '83 min', '92 min', '127 min', '139 min',
       '79 min', '192 min', '131 min', '123 min', '122 min', '82 min',
       '10 seasons',

In [67]:
#Hay nulos y los otros valores estan todos normales. Hacemos la division.
divisionDuration(hulu_titles)
hulu_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration.int,duration.type,listed_in,description
0,hs1,movie,ricky velez: here's everything,NaN,NaN,NaN,2021-10-24,2021,tv-ma,NaN,NaN,NaN,"comedy, stand up",​comedian ricky velez bares it all with his ho...
1,hs2,movie,silent night,NaN,NaN,NaN,2021-10-23,2020,g,94 min,94.0,min,"crime, drama, thriller","mark, a low end south london hitman recently r..."
2,hs3,movie,the marksman,NaN,NaN,NaN,2021-10-23,2021,pg-13,108 min,108.0,min,"action, thriller",a hardened arizona rancher tries to protect an...
3,hs4,movie,gaia,NaN,NaN,NaN,2021-10-22,2021,r,97 min,97.0,min,horror,a forest ranger and two survivalists with a cu...
4,hs5,movie,settlers,NaN,NaN,NaN,2021-10-22,2021,g,104 min,104.0,min,"science fiction, thriller",mankind's earliest settlers on the martian fro...


In [68]:
#Quitamos la columna duration
hulu_titles.drop(columns='duration',inplace=True)

In [69]:
hulu_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        3073 non-null   object        
 1   type           3073 non-null   object        
 2   title          3073 non-null   object        
 3   director       3 non-null      object        
 4   cast           0 non-null      float64       
 5   country        1620 non-null   object        
 6   date_added     3045 non-null   datetime64[ns]
 7   release_year   3073 non-null   int64         
 8   rating         3073 non-null   object        
 9   duration.int   2886 non-null   float64       
 10  duration.type  2886 non-null   object        
 11  listed_in      3073 non-null   object        
 12  description    3069 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 312.2+ KB


In [70]:
#Observamos los valores que quedan en duration.type
hulu_titles['duration.type'].unique()

array([nan, 'min', 'season', 'seasons'], dtype=object)

In [71]:
#Unificamos Seasons y Season.
hulu_titles['duration.type'].replace(to_replace='season',value='seasons',inplace=True)
hulu_titles['duration.type'].unique()

array([nan, 'min', 'seasons'], dtype=object)

In [72]:
#Ahora lo hacemos con netflix.
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [73]:
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [74]:
#Vemos que no hay nulos bajo show_id. Controlamos que no se repitan.
controlRepetidos(netflix_titles,'show_id')

Se repiten 0 valores mas de una vez


In [75]:
#Agregamos la letra en show_id.
agregarLetraId(netflix_titles,'n')
netflix_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\1232259299.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['show_id'][i]=letra + dataframe['show_id'][i]


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ns1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,ns2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,ns3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,ns4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,ns5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [76]:
#Vemos que tipo de valores hay en la columna rating
netflix_titles['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR', nan,
       'TV-Y7-FV', 'UR'], dtype=object)

In [77]:
#Examinamos todas las que tienen min.
duracionEnRating(netflix_titles,'min')

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,ns5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,ns5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,ns5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...


In [78]:
#Son 3 valores. Reemplazamos los valores y chequeamos que se hayan hecho los cambios.
cambioDeColumnas(netflix_titles,'rating','duration','min')
lista_ids = ['ns5542','ns5795','ns5814']
netflix_titles[netflix_titles['show_id'].isin(lista_ids)]

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_hacia][i] = dataframe[campo_desde][i]
C:\Users\larra\AppData\Local\Temp\ipykernel_5372\4002361211.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[campo_desde][i] = aux


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,ns5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,NaN,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,ns5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,NaN,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,ns5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,NaN,66 min,Movies,The comic puts his trademark hilarious/thought...


In [79]:
#Vemos nuevamente que tipo de valores hay en la columna rating
netflix_titles['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', nan, 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

In [80]:
#Nuevamente. Debiera normalizarse y elegir uno solo. De momento solo se llena con G.
netflix_titles['rating'].fillna('G',inplace=True)
netflix_titles[netflix_titles['show_id'].isin(lista_ids)]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
5541,ns5542,Movie,Louis C.K. 2017,Louis C.K.,Louis C.K.,United States,"April 4, 2017",2017,G,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi..."
5794,ns5795,Movie,Louis C.K.: Hilarious,Louis C.K.,Louis C.K.,United States,"September 16, 2016",2010,G,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...
5813,ns5814,Movie,Louis C.K.: Live at the Comedy Store,Louis C.K.,Louis C.K.,United States,"August 15, 2016",2015,G,66 min,Movies,The comic puts his trademark hilarious/thought...


In [81]:
#Vemos que se encuentra bajo la columna date_added.
netflix_titles['date_added'].value_counts()

January 1, 2020      109
November 1, 2019      89
March 1, 2018         75
December 31, 2019     74
October 1, 2018       71
                    ... 
December 4, 2016       1
November 21, 2016      1
November 19, 2016      1
November 17, 2016      1
January 11, 2020       1
Name: date_added, Length: 1767, dtype: int64

In [82]:
#Vemos cuantas son.
netflix_titles['date_added'].unique().size

1768

In [83]:
#Modificamos el formato de las fechas.
netflix_titles = netflix_titles.astype({'date_added':'datetime64[D]'})
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ns1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,ns2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,ns3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,ns4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,ns5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [84]:
#Vemos que se haya modificado.
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   show_id       8807 non-null   object        
 1   type          8807 non-null   object        
 2   title         8807 non-null   object        
 3   director      6173 non-null   object        
 4   cast          7982 non-null   object        
 5   country       7976 non-null   object        
 6   date_added    8797 non-null   datetime64[ns]
 7   release_year  8807 non-null   int64         
 8   rating        8807 non-null   object        
 9   duration      8807 non-null   object        
 10  listed_in     8807 non-null   object        
 11  description   8807 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 825.8+ KB


In [85]:
#Corroboramos que coincida.
print(netflix_titles['date_added'].unique().size)
print(netflix_titles['date_added'].unique())

1715
['2021-09-25T00:00:00.000000000' '2021-09-24T00:00:00.000000000'
 '2021-09-23T00:00:00.000000000' ... '2018-12-06T00:00:00.000000000'
 '2016-03-09T00:00:00.000000000' '2020-01-11T00:00:00.000000000']


In [86]:
#Modificamos las columnas de texto a minusculas.
textoEnMinusculas(netflix_titles)
netflix_titles.head()

C:\Users\larra\AppData\Local\Temp\ipykernel_5372\3644383547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[col][i] = dataframe[col][i].casefold()


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,ns1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm..."
1,ns2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t..."
2,ns3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...
3,ns4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo..."
4,ns5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...


In [87]:
#Vemos que hay debajo del campo duration
netflix_titles['duration'].unique()

array(['90 min', '2 seasons', '1 season', '91 min', '125 min',
       '9 seasons', '104 min', '127 min', '4 seasons', '67 min', '94 min',
       '5 seasons', '161 min', '61 min', '166 min', '147 min', '103 min',
       '97 min', '106 min', '111 min', '3 seasons', '110 min', '105 min',
       '96 min', '124 min', '116 min', '98 min', '23 min', '115 min',
       '122 min', '99 min', '88 min', '100 min', '6 seasons', '102 min',
       '93 min', '95 min', '85 min', '83 min', '113 min', '13 min',
       '182 min', '48 min', '145 min', '87 min', '92 min', '80 min',
       '117 min', '128 min', '119 min', '143 min', '114 min', '118 min',
       '108 min', '63 min', '121 min', '142 min', '154 min', '120 min',
       '82 min', '109 min', '101 min', '86 min', '229 min', '76 min',
       '89 min', '156 min', '112 min', '107 min', '129 min', '135 min',
       '136 min', '165 min', '150 min', '133 min', '70 min', '84 min',
       '140 min', '78 min', '7 seasons', '64 min', '59 min', '139 min',
    

In [88]:
#No hay nulos y los valores estan todos normales. Hacemos la division.
divisionDuration(netflix_titles)
netflix_titles.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,duration.int,duration.type,listed_in,description
0,ns1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,90,min,documentaries,"as her father nears the end of his life, filmm..."
1,ns2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,2,seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t..."
2,ns3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,1,season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...
3,ns4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,1,season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo..."
4,ns5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,2,seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...


In [89]:
#Quitamos la columna duration
netflix_titles.drop(columns='duration',inplace=True)

In [90]:
#Observamos los valores que quedan en duration.type
netflix_titles['duration.type'].unique()

array(['min', 'seasons', 'season'], dtype=object)

In [91]:
#Unificamos Seasons y Season.
netflix_titles['duration.type'].replace(to_replace='season',value='seasons',inplace=True)
netflix_titles['duration.type'].unique()

array(['min', 'seasons'], dtype=object)

In [92]:
#Vemos que todos los cambios se hayan hecho correctamente
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   show_id        8807 non-null   object        
 1   type           8807 non-null   object        
 2   title          8807 non-null   object        
 3   director       6173 non-null   object        
 4   cast           7982 non-null   object        
 5   country        7976 non-null   object        
 6   date_added     8797 non-null   datetime64[ns]
 7   release_year   8807 non-null   int64         
 8   rating         8807 non-null   object        
 9   duration.int   8807 non-null   int64         
 10  duration.type  8807 non-null   object        
 11  listed_in      8807 non-null   object        
 12  description    8807 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(10)
memory usage: 894.6+ KB


Ahora debieramos importar los datasets de los rankings. Vamos a examinar primero el 1 y luego vemos como continuamos.

In [93]:
#Importamos los datos de dataset 1
rank = pd.read_csv('C:/Users/larra/OneDrive/Documents/HENRY/PI01/MLOpsReviews/ratings/1.csv')
rank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   userId     1500000 non-null  int64  
 1   rating     1500000 non-null  float64
 2   timestamp  1500000 non-null  int64  
 3   movieId    1500000 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 45.8+ MB


In [94]:
#Si todos los datasets contienen 1.500.000 registros serian 12.000.000 de registros. Los ponemos todos en un solo dataframe
rankings = pd.read_csv('https://drive.google.com/uc?id=1bXWiH1tqXpn73runZLGOjfqUYrRWIuzi')
rankings = pd.concat([rankings,
                      pd.read_csv('https://drive.google.com/uc?id=1LskyEBGK-DCHnrqbOIYqc8A2AzRoH0fy'),
                      pd.read_csv('https://drive.google.com/uc?id=1sHwR3CHXe4s0RSlcq3c83Ix12pxVZ6Yv'),
                      pd.read_csv('https://drive.google.com/uc?id=1jYUi_spkJYbbPJ5XuRKtf8I_DKcaP95N'),
                      pd.read_csv('https://drive.google.com/uc?id=1Ta8qx0kYkInr5KURq1A8gyfFqNqcQWGZ'),
                      pd.read_csv('https://drive.google.com/uc?id=1ZdEoc0n5-r44wXNBKfXmaO76NN9P5ErP'),
                      pd.read_csv('https://drive.google.com/uc?id=1ValM2b4d1u1oyrL3QqAkJMLkb4kJLNzo'),
                      pd.read_csv('https://drive.google.com/uc?id=13NU2TFaLksX-cG10R63u6xGtHxz69vlW'),],axis=0)

Se agregaron  1500000  registros
Se agregaron  1500000  registros
Se agregaron  1500000  registros
Se agregaron  1500000  registros
Se agregaron  1500000  registros
Se agregaron  1500000  registros
Se agregaron  524289  registros
Se agregaron  1500000  registros


In [95]:
#Lo revisamos
rankings.head()

,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186
2,1,5.0,1425941523,hs2381
3,1,5.0,1425941546,ns3663
4,1,5.0,1425941556,as9500


In [96]:
rankings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 420.5+ MB


Vamos a pasar timestamp a un formato de fecha.

In [97]:
#Empezamos transformandolo a datetime. 
rankings.insert(loc=2,column='date_col',value=pd.to_datetime(rankings['timestamp'],unit='s'))
rankings.head()

,userId,rating,date_col,timestamp,movieId
0,1,1.0,2015-03-09 22:52:09,1425941529,as680
1,1,4.5,2015-03-09 23:07:15,1425942435,ns2186
2,1,5.0,2015-03-09 22:52:03,1425941523,hs2381
3,1,5.0,2015-03-09 22:52:26,1425941546,ns3663
4,1,5.0,2015-03-09 22:52:36,1425941556,as9500


In [98]:
#Vemos que formato tiene
rankings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 5 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int64         
 1   rating     float64       
 2   date_col   datetime64[ns]
 3   timestamp  int64         
 4   movieId    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 504.7+ MB


In [99]:
#Lo volvemos a pasar a string pero ahora truncado
rankings.insert(loc=2,column='date',value=rankings['date_col'].dt.date.astype(str))
rankings.head()

,userId,rating,date,date_col,timestamp,movieId
0,1,1.0,2015-03-09,2015-03-09 22:52:09,1425941529,as680
1,1,4.5,2015-03-09,2015-03-09 23:07:15,1425942435,ns2186
2,1,5.0,2015-03-09,2015-03-09 22:52:03,1425941523,hs2381
3,1,5.0,2015-03-09,2015-03-09 22:52:26,1425941546,ns3663
4,1,5.0,2015-03-09,2015-03-09 22:52:36,1425941556,as9500


In [100]:
#Quitamos las columnas que ya no nos sirven.
rankings.drop(columns=['timestamp','date_col'],inplace=True)

In [101]:
#Vemos que nos queda.
rankings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   rating   float64
 2   date     object 
 3   movieId  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 420.5+ MB


In [102]:
#Convertimos la columna date a formato fecha y revisamos.
rankings = rankings.astype({'date':'datetime64[D]'})
rankings.head()

,userId,rating,date,movieId
0,1,1.0,2015-03-09,as680
1,1,4.5,2015-03-09,ns2186
2,1,5.0,2015-03-09,hs2381
3,1,5.0,2015-03-09,ns3663
4,1,5.0,2015-03-09,as9500


In [103]:
rankings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column   Dtype         
---  ------   -----         
 0   userId   int64         
 1   rating   float64       
 2   date     datetime64[ns]
 3   movieId  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 420.5+ MB


Desarrollo API: Propones disponibilizar los datos de la empresa usando el framework FastAPI. Las consultas que propones son las siguientes:

In [119]:
#Para hacer las funciones vamos a unir todos los titulos en un solo dataframe y a tirar las columnas que no se usen en las funciones.
plataformas_API = pd.concat([amazon_titles,netflix_titles,disney_titles,hulu_titles])
plataformas_API.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration.int,duration.type,listed_in,description
0,as1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,113.0,min,"comedy, drama",a small fishing village must procure a local d...
1,as2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,110.0,min,"drama, international",a metro family decides to fight a cyber crimin...
2,as3,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,74.0,min,"action, drama, suspense",after a man discovers his wife is cheating on ...
3,as4,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,69.0,min,documentary,"pink breaks the mold once again, bringing her ..."
4,as5,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,45.0,min,"drama, fantasy",teenage matt banting wants to work with a famo...


In [120]:
#Descartamos las que no se usan en ninguna función.
plataformas_API.drop(columns=['director','country','date_added','rating','listed_in','description'],inplace=True)
plataformas_API.head()

,show_id,type,title,cast,release_year,duration.int,duration.type
0,as1,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2014,113.0,min
1,as2,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2018,110.0,min
2,as3,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2017,74.0,min
3,as4,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2014,69.0,min
4,as5,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",1989,45.0,min


In [129]:
#Además le vamos a agregar una columna con el promedio de los puntajes.
prom_rating = rankings.drop(columns=['userId','date']).groupby(by='movieId').mean()
prom_rating.head()

,rating
movieId,
as1,3.467131
as10,3.439571
as100,3.609302
as1000,3.556701
as1001,3.585288


In [132]:
plataformas_API = pd.merge(plataformas_API,prom_rating,left_on='show_id',right_on='movieId')
plataformas_API.head(10)

,show_id,type,title,cast,release_year,duration.int,duration.type,rating
0,as1,movie,the grand seduction,"brendan gleeson, taylor kitsch, gordon pinsent",2014,113.0,min,3.467131
1,as2,movie,take care good night,"mahesh manjrekar, abhay mahajan, sachin khedekar",2018,110.0,min,3.548682
2,as3,movie,secrets of deception,"tom sizemore, lorenzo lamas, robert lasardo, r...",2017,74.0,min,3.500000
3,as4,movie,pink: staying true,"interviews with: pink, adele, beyoncé, britney...",2014,69.0,min,3.538055
4,as5,movie,monster maker,"harry dean stanton, kieran o'brien, george cos...",1989,45.0,min,3.478992
5,as6,movie,living with dinosaurs,"gregory chisholm, juliet stevenson, brian hens...",1989,52.0,min,3.555195
6,as7,movie,hired gun,"alice cooper, liberty devitto, ray parker jr.,...",2017,98.0,min,3.553232
7,as8,movie,grease live!,"julianne hough, aaron tveit, vanessa hudgens, ...",2016,131.0,min,3.517274
8,as9,movie,global meltdown,"michael paré, leanne khol young, patrick j. ma...",2017,87.0,min,3.501012
9,as10,movie,david's mother,"kirstie alley, sam waterston, stockard channing",1994,92.0,min,3.439571


- Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))

In [104]:
#Realizamos una funcion auxiliar para no repetir lineas de código dentro de la función principal.
def maxDuracion(dataframe,anio,tipo_dur):
    if tipo_dur.find('eas')!=-1:
        tipo_dur = 'seasons'
    else:
        tipo_dur = 'min'
    cond1 = dataframe['release_year']==anio
    cond2 = dataframe['duration.type']==tipo_dur
    df_filtrado = dataframe[cond1 & cond2]
    return df_filtrado[df_filtrado['duration.int']==df_filtrado['duration.int'].max()]['title'].values[0]

In [125]:
#Realizamos la funcion principal
def get_max_duration(year, platform, duration_type):
    platform = platform.casefold()
    if platform.find('net') != -1:
        return maxDuracion(plataformas_API[plataformas_API['show_id'].str.contains('ns')],year,duration_type)
    elif platform.find('hul') != -1:
        return maxDuracion(plataformas_API[plataformas_API['show_id'].str.contains('hs')],year,duration_type)
    elif platform.find('disn') != -1:
        return maxDuracion(plataformas_API[plataformas_API['show_id'].str.contains('ds')],year,duration_type)
    elif platform.find('amaz') != -1:
        return maxDuracion(plataformas_API[plataformas_API['show_id'].str.contains('as')],year,duration_type)

In [148]:
get_max_duration(2015,'amazon','season')

'two and a half men'

In [157]:
cond1 = plataformas_API['show_id'].str.contains('as')
cond2 = plataformas_API['release_year']==2015
cond3 = plataformas_API['duration.type']=='seasons'
df = plataformas_API[cond1&cond2&cond3]
df[df['duration.int']==df['duration.int'].max()]['title'].values

array(['two and a half men'], dtype=object)


Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))

In [135]:
#Realizamos una funcion auxiliar para no repetir lineas de código dentro de la función principal.
def puntajes(dataframe,anio,puntaje):
    cond1 = dataframe['release_year']==anio
    cond2 = dataframe['rating']>=puntaje
    df_filtrado = dataframe[cond1 & cond2]
    return df_filtrado.shape[0]

In [134]:
#Hacemos la funcion principal
def get_score_count(platform, scored, year):
    platform = platform.casefold()
    if platform.find('net') != -1:
        return puntajes(plataformas_API[plataformas_API['show_id'].str.contains('ns')],year,scored)
    elif platform.find('hul') != -1:
        return puntajes(plataformas_API[plataformas_API['show_id'].str.contains('hs')],year,scored)
    elif platform.find('disn') != -1:
        return puntajes(plataformas_API[plataformas_API['show_id'].str.contains('ds')],year,scored)
    elif platform.find('amaz') != -1:
        return puntajes(plataformas_API[plataformas_API['show_id'].str.contains('as')],year,scored)


In [108]:
'''Dejo la funcion para consultar peliculas con un puntaje mayor a XX que fueron puntuadas
en un determinado año (asi interpreté en un primer momento)'''
def get_score_count2(platform, scored, year):
    platform = platform.casefold()
    if platform.find('net') != -1:
        cond1 = rankings['movieId'].str.contains('ns')
    elif platform.find('hul') != -1:
        cond1 = rankings['movieId'].str.contains('hs')
    elif platform.find('disn') != -1:
        cond1 = rankings['movieId'].str.contains('ds')
    elif platform.find('amaz') != -1:
        cond1 = rankings['movieId'].str.contains('as')
    
    cond2 = rankings['date'].dt.year==year
    dataframe = rankings[cond1 & cond2].drop(columns='userId')
    dataframe = dataframe.groupby('movieId').mean()
    return int(dataframe[dataframe['rating']>scored].shape[0])

In [158]:
#Hacemos una prueba
get_score_count('amazon',3.3,2015)

378

In [140]:
#Comprobamos
cond1 = plataformas_API['show_id'].str.contains('ns')
cond2 = plataformas_API['release_year']==2015
cond3 = plataformas_API['rating']>=3.3
plataformas_API[cond1&cond2&cond3]

,show_id,type,title,cast,release_year,duration.int,duration.type,rating
9737,ns70,tv show,stories by rabindranath tagore,NaN,2015,1.0,seasons,3.508000
9784,ns117,movie,dhanak,"krrish chhabria, hetal gada, vipin sharma, gul...",2015,114.0,min,3.452479
9801,ns134,movie,chappie,"sharlto copley, hugh jackman, sigourney weaver...",2015,121.0,min,3.527897
9809,ns142,movie,extraction,"bruce willis, kellan lutz, gina carano, d.b. s...",2015,82.0,min,3.536346
9822,ns155,tv show,kuroko's basketball,"kensho ono, yuki ono, chiwa saito, yoshimasa h...",2015,3.0,seasons,3.542553
...,...,...,...,...,...,...,...,...
18452,ns8785,movie,yoko and his friends,"eileen stevens, alyson leigh rosenfeld, sarah ...",2015,78.0,min,3.525000
18456,ns8789,movie,you carry me,"lana baric, vojislav brajovic, natasa janjic, ...",2015,157.0,min,3.554147
18463,ns8796,tv show,yu-gi-oh! arc-v,"mike liscio, emily bauer, billy bob thompson, ...",2015,2.0,seasons,3.534413
18469,ns8802,movie,zinzana,"ali suliman, saleh bakri, yasa, ali al-jabri, ...",2015,96.0,min,3.573404


Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [143]:
#Definimos la funcion
def get_count_platform(platform):
    platform = platform.casefold()
    if platform.find('net') != -1:
        return plataformas_API[plataformas_API['show_id'].str.contains('ns')].shape[0]
    elif platform.find('hul') != -1:
        return plataformas_API[plataformas_API['show_id'].str.contains('hs')].shape[0]
    elif platform.find('disn') != -1:
        return plataformas_API[plataformas_API['show_id'].str.contains('ds')].shape[0]
    elif platform.find('amaz') != -1:
        return plataformas_API[plataformas_API['show_id'].str.contains('as')].shape[0]

In [159]:
#Comprobamos
get_count_platform('amazon')

9668


Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))

In [115]:
#Definimos la funcion que encuentra el actor que mas se repite en una plataforma especifica
def actorMayorPresencia(dataframe,anio):
    actores = []
    cast = dataframe[dataframe['release_year']==anio]['cast']
    for i in cast:
        try:
            if i.find(',')!=-1:
                for act in i.split(','):
                    actores.append(act.strip().title())
            else:
                actores.append(i.strip().title())
        except:
            pass
    return max(set(actores),key=actores.count)

In [145]:
#Definimos la funcion principal en la que dividimos por plataforma.
def get_actor(platform, year):
    platform = platform.casefold()
    if platform.find('net') != -1:
        return actorMayorPresencia(plataformas_API[plataformas_API['show_id'].str.contains('ns')],year)
    elif platform.find('hul') != -1:
        return actorMayorPresencia(plataformas_API[plataformas_API['show_id'].str.contains('hs')],year)
    elif platform.find('disn') != -1:
        return actorMayorPresencia(plataformas_API[plataformas_API['show_id'].str.contains('ds')],year)
    elif platform.find('amaz') != -1:
        return actorMayorPresencia(plataformas_API[plataformas_API['show_id'].str.contains('as')],year)

In [146]:
#Verificamos
get_actor('netflix',1990)

'Michael Ontkean'

In [147]:
#Exportamos el dataframe así lo usamos en la API
plataformas_API.to_csv('plataformas_API.csv',index=False)